In [16]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
# Additional libraries for data handling, preprocessing, etc.


In [17]:
# Define input shape based on your image size (here, 640x360)
input_shape = (360, 640, 3)

# Load MobileNet, excluding the final classification layer
mobilenet = MobileNet(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze the pre-trained layers to avoid retraining them (optional)
for layer in mobilenet.layers:
  layer.trainable = False


C:\Users\ukrst\AppData\Local\Temp\ipykernel_9696\1475731864.py:5: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  mobilenet = MobileNet(weights='imagenet', include_top=False, input_shape=input_shape)


In [25]:
# GlobalAveragePooling2D reduces spatial dimensions to a single vector
x = GlobalAveragePooling2D()(mobilenet.output)

# Dense layer with 4 units (one for each variant) and softmax activation
predictions = Dense(4, activation='sigmoid')(x)

# Create the final model
model = tf.keras.Model(inputs=mobilenet.input, outputs=predictions)


In [26]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [20]:
import cv2
import os
import numpy as np

# Define empty lists for images and labels
X = []
y = []

# Folder path (replace 'path/to/images' with your actual folder path)
folder_path = 'classification_data/'

# Loop through folders
for folder_name in ['attack_img', 'explore_img', 'loot_img', 'getaway_img']:
    folder_full_path = os.path.join(folder_path + folder_name)
    
    class_label = {
      'attack_img': [1,0,0,0],
      'explore_img': [0,1,0,0],
      'loot_img': [0,0,1,0],
      'getaway_img': [0,0,0,1]
    }[folder_name]
    
    for filename in os.listdir(folder_full_path):
        if filename.endswith(".jpg") :
            img = cv2.imread(os.path.join(folder_full_path, filename))
        
            # Append image to X and label to y
            X.append(img)
            
            
            y.append(class_label)
        

X = np.asarray(X)
y = np.asarray(y)

In [21]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)

(396, 360, 640, 3) (396, 4)


In [27]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))


Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 37s 3s/step - accuracy: 0.4046 - loss: 1.6024 - val_accuracy: 0.6300 - val_loss: 1.2996
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 31s 2s/step - accuracy: 0.6880 - loss: 1.0174 - val_accuracy: 0.6300 - val_loss: 0.9874
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 33s 3s/step - accuracy: 0.6711 - loss: 0.9593 - val_accuracy: 0.6400 - val_loss: 1.0118
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 33s 3s/step - accuracy: 0.6457 - loss: 0.9780 - val_accuracy: 0.6500 - val_loss: 0.9339
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - accuracy: 0.6585 - loss: 0.9411 - val_accuracy: 0.6500 - val_loss: 0.9207
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 31s 2s/step - accuracy: 0.6427 - loss: 0.9157 - val_accuracy: 0.6500 - val_loss: 0.8986
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 32s 3s/step - accuracy: 0.6575 - loss: 0.8849 - val_accuracy: 0.6300 - val_loss: 0.9160
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - accuracy: 0.6837 - loss: 0.8204 - val_accuracy: 0.6600 - val_loss:

KeyboardInterrupt: 

In [23]:
loss, accuracy = model.evaluate(X_val, y_val)
print('Test accuracy:', accuracy)

model.save('relu_test.h5')

4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.6353 - loss: 4.4623


Test accuracy: 0.6299999952316284


In [29]:
model_rap = tf.keras.models.load_model('class_2.h5')


for n in range(20):
    y_real = y_val[n]
    tbp = np.asarray(X_val[n])
    cv2.imshow('ss', tbp)
    tbp = np.expand_dims(tbp, axis=0)
    predictions = model_rap.predict(tbp)
    print(predictions, "\n", y_real)

    cv2.waitKey(0) & 0xFF
cv2.destroyWindow('ss')

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 951ms/step
[[0.87198615 0.02390181 0.00413186 0.09998013]] 
 [0 0 0 1]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
[[0.8573707  0.01343981 0.01004975 0.11913973]] 
 [1 0 0 0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
[[0.7301812  0.10763477 0.0070095  0.15517454]] 
 [1 0 0 0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
[[0.68777764 0.07467528 0.01909654 0.21845046]] 
 [1 0 0 0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
[[0.7105981  0.22148296 0.01429841 0.05362042]] 
 [1 0 0 0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
[[0.13467148 0.17349307 0.00265746 0.689178  ]] 
 [0 0 0 1]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
[[0.5976322  0.06744435 0.01315348 0.3217699 ]] 
 [1 0 0 0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
[[0.8227104  0.05339312 0.02025013 0.10364626]] 
 [1 0 0 0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
[[0.9335157  0.01506054 0.0058881  0.04553559]] 
 [1 0 0 0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
[[0.85788465 0.04079035 0.00541043 0.09591464]] 
 [1 0 0 0]
1/1 ━━━━━━━━━━━━━━━

In [28]:
model_rap = tf.keras.models.load_model('my_mobilenet_model.h5')
model_rap.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model_rap.fit(X_train, y_train, epochs=40, validation_data=(X_val, y_val))
model_rap.save('class_2.h5')

Epoch 1/40
13/13 ━━━━━━━━━━━━━━━━━━━━ 34s 2s/step - accuracy: 0.7854 - loss: 0.5956 - val_accuracy: 0.7400 - val_loss: 0.7340
Epoch 2/40
13/13 ━━━━━━━━━━━━━━━━━━━━ 27s 2s/step - accuracy: 0.8160 - loss: 0.5927 - val_accuracy: 0.7000 - val_loss: 0.7350
Epoch 3/40
13/13 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - accuracy: 0.7564 - loss: 0.6643 - val_accuracy: 0.6900 - val_loss: 0.7348
Epoch 4/40
13/13 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - accuracy: 0.7663 - loss: 0.5880 - val_accuracy: 0.7200 - val_loss: 0.7174
Epoch 5/40
13/13 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - accuracy: 0.7939 - loss: 0.5774 - val_accuracy: 0.7500 - val_loss: 0.6913
Epoch 6/40
13/13 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - accuracy: 0.7758 - loss: 0.5911 - val_accuracy: 0.7700 - val_loss: 0.6796
Epoch 7/40
13/13 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - accuracy: 0.8515 - loss: 0.5431 - val_accuracy: 0.7200 - val_loss: 0.6956
Epoch 8/40
13/13 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - accuracy: 0.8156 - loss: 0.5335 - val_accuracy: 0.7600 - val_loss: